In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import dataprob
import copy

import linkage


In [ ]:
df = pd.DataFrame({"out":[1,2]})
issubclass(type(None),type(None))


#### Load experimental data

In [ ]:

blank = linkage.experiment.Experiment("data/itc_blank_expt.csv",
                                      cell_contents={},
                                      syringe_contents={"ET":5e-3},
                                      cell_volume=280)
blank.define_itc_observable(obs_column="heat",
                            obs_stdev=0.01)

binding = linkage.experiment.Experiment("data/itc_binding_expt.csv",
                                        cell_contents={"CT":0.5e-3},
                                        syringe_contents={"ET":5e-3},
                                        cell_volume=280)
binding.define_itc_observable(obs_column="heat",
                              obs_stdev=0.01)


#### Create a linkage model

In [ ]:
expt_list = [blank,binding] 

gm = linkage.GlobalModel(model_name="CaEDTA",
                         expt_list=expt_list)

In [ ]:
from dataprob.model_wrapper import vector_model_wrapper
fittable_params = dict(zip(gm.parameter_names,gm.parameter_guesses))
mw = vector_model_wrapper.VectorModelWrapper(gm.model_normalized,
                                             fittable_params=fittable_params)

#### Plot data

In [ ]:
mw.fit_parameters

In [ ]:
style = {"s":50,
         "facecolor":"none",
         "edgecolor":"black"}

color_order = ["red","black"]
fig, ax = plt.subplots(1,figsize=(6,6))
for i in range(len(gm._expt_list)):
    style["edgecolor"] = color_order[i]
    ax.scatter(np.arange(len(gm._expt_list[i].expt_data["heat"])),
               gm._expt_list[i].expt_data["heat"],**style)
plt.xlabel("injection")
plt.ylabel("heat")


#### Do fit

In [ ]:
help(gm.model_normalized)

In [ ]:
mw = dataprob.ModelWrapper(gm.model_normalized,
                           fittable_params=gm.parameter_names)
mw.guesses = gm.parameter_guesses

In [ ]:
#BayesianFitter does a Bayesian sampling rather than maximum likelihood fit

F = dataprob.MLFitter()
#F = dataprob.BayesianFitter()
F.fit(model=mw, #gm.model_normalized,
      guesses=[5,-11000,0,0],
      #names=gm.parameter_names,
      y_obs=gm.y_obs_normalized,
      y_stdev=gm.y_stdev_normalized)
F.fit_df

#### Plot results

In [ ]:

df = gm.as_df

style = {"s":50,
         "facecolor":"none",
         "edgecolor":"black"}
color_order = ["red","black"]

for expt in np.unique(df["expt_id"]):
    this_df = df.loc[df["expt_id"] == expt,:]

    x = np.arange(len(this_df))

    style["edgecolor"] = color_order[expt]
    plt.scatter(x,this_df["y_obs"],**style)
    plt.errorbar(x=x,
                 y=this_df["y_obs"],
                 yerr=this_df["y_stdev"],
                 ls="none",
                 lw=1,
                 capsize=3,
                 color=color_order[expt])
    plt.plot(np.arange(len(this_df)),
             this_df["y_calc"],
             '-',
             lw=2,
             color=color_order[expt])
plt.xlabel("injection")
plt.ylabel("heat")

F.fit_df

#### Corner plot to look for parameter correlations

In [ ]:
_ = F.corner_plot(filter_params="nuisance")